In [3]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import ensemble, metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib 
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report

Using matplotlib backend: Qt5Agg


In [4]:
df = pd.read_csv("train.csv", decimal = '.') #Ввод данных
df = df.drop(columns='ID_FIRM') #Удаление столбца с ID
test = df['BANKR'] #Данные для проверки

In [4]:
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100))) #Проверка пропущенных значений

ID_FIRM - 0%
P1110_B - 0%
P1110_E - 0%
P1120_B - 0%
P1120_E - 0%
P1130_B - 0%
P1130_E - 0%
P1140_B - 0%
P1140_E - 0%
P1150_B - 0%
P1150_E - 0%
P1160_B - 0%
P1160_E - 0%
P1170_B - 0%
P1170_E - 0%
P1180_B - 0%
P1180_E - 0%
P1190_B - 0%
P1190_E - 0%
P1100_B - 0%
P1100_E - 0%
P1210_B - 0%
P1210_E - 0%
P1220_B - 0%
P1220_E - 0%
P1230_B - 0%
P1230_E - 0%
P1240_B - 0%
P1240_E - 0%
P1250_B - 0%
P1250_E - 0%
P1260_B - 0%
P1260_E - 0%
P1200_B - 0%
P1200_E - 0%
P1600_B - 0%
P1600_E - 0%
P1310_B - 0%
P1310_E - 0%
P1320_B - 0%
P1320_E - 0%
P1340_B - 0%
P1340_E - 0%
P1350_B - 0%
P1350_E - 0%
P1360_B - 0%
P1360_E - 0%
P1370_B - 0%
P1370_E - 0%
P1300_B - 0%
P1300_E - 0%
P1410_B - 0%
P1410_E - 0%
P1420_B - 0%
P1420_E - 0%
P1430_B - 0%
P1430_E - 0%
P1450_B - 0%
P1450_E - 0%
P1400_B - 0%
P1400_E - 0%
P1510_B - 0%
P1510_E - 0%
P1520_B - 0%
P1520_E - 0%
P1530_B - 0%
P1530_E - 0%
P1540_B - 0%
P1540_E - 0%
P1550_B - 0%
P1550_E - 0%
P1500_B - 0%
P1500_E - 0%
P1700_B - 0%
P1700_E - 0%
P2110_B - 0%
P2110_E - 0%

In [15]:
desc = df.describe()
writers = pd.ExcelWriter('Describe.xlsx')
desc.to_excel(writers,'AR')
writers.save()

In [17]:
cor = df.corr() #Просмотр зависимостей признаков
writers = pd.ExcelWriter('Correlation.xlsx')
cor.to_excel(writers,'AR')
writers.save()

In [59]:
df['BANKR'].value_counts() #Вывод кол-ва кажого класса

0    730141
1      3130
Name: BANKR, dtype: int64

In [43]:
X = df.drop(('BANKR'),axis=1)
Y = df['BANKR']
os = SMOTE(random_state=0) 
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=11)
X_train, Y_train = os.fit_resample(X_train, Y_train) #Добавление синтетических данных, из-за сильного превосходства одного класса над другим
N_train, _ = X_train.shape
N_test, _ = X_test.shape
columns = X_train.columns
print(N_train, N_test) #Разбивка датасета на обучающий и тестовый

1095152 183318


In [45]:
X_train1 =pd.DataFrame(data = X_train, columns = columns)
Y_train1 =pd.DataFrame(data = Y_train, columns=['y'])
print("length of oversampled data is ",len(X_train1)) #Вывод кол-ва синтетических данных

length of oversampled data is  1095152


In [44]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) #Нормализация признаков

In [ ]:
# Hyperparameter grid
param_grid = {
    'n_estimators': np.linspace(10, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# Estimator for use in random search
estimator = ensemble.RandomForestClassifier(random_state = 11)

# Create the random search model
rs = RandomizedSearchCV(estimator, param_grid, n_jobs = -1, 
                        scoring = 'roc_auc', cv = 3, 
                        n_iter = 10, verbose = 1, random_state=11)
rs.fit(X_train,Y_train) #Выявление оптимальных параметров для обучения модели случайного леса

In [ ]:
model = ensemble.GradientBoostingClassifier(random_state = 11)
parameters = {'learning_rate': sp_randFloat(),
                  'subsample'    : sp_randFloat(),
                  'n_estimators' : sp_randInt(10, 200),
                  'max_depth'    : sp_randInt(4, 10) 
                 }
    
randm = RandomizedSearchCV(model, parameters, n_jobs = -1, 
                        scoring = 'roc_auc', cv = 3, 
                        n_iter = 10, verbose = 1, random_state=11)
randm.fit(X_train, Y_train) #Выявление оптимальных параметров для обучения модели градиентного бустинга

In [7]:
randm.best_params_ #Вывод оптимальных параметров

{'learning_rate': 0.15791230667493694,
 'max_depth': 9,
 'n_estimators': 89,
 'subsample': 0.8182753578641435}

In [34]:
rs.best_params_ #Вывод оптимальных параметров

{'n_estimators': 114,
 'min_samples_split': 2,
 'max_leaf_nodes': 44,
 'max_features': 'auto',
 'max_depth': 9,
 'bootstrap': False}

In [46]:
rf = ensemble.RandomForestClassifier(n_estimators = 114,random_state =11,n_jobs=-1, min_samples_split = 2, max_leaf_nodes = 44, max_depth = 9, bootstrap = False)
rf.fit(X_train, Y_train)
err_train = np.mean(Y_train != rf.predict(X_train))
err_test = np.mean(Y_test != rf.predict(X_test))
print(err_train,err_test) #Обучение модели и выявление процента ошибки

0.1386739009744766 0.14342290446110037


In [13]:
gbt = ensemble.GradientBoostingClassifier(n_estimators=89, random_state=11, subsample = 0.818275, max_depth = 9, learning_rate = 0.157912)
gbt.fit(X_train, Y_train)
err_train = np.mean(Y_train != gbt.predict(X_train))
err_test = np.mean(Y_test != gbt.predict(X_test))
print(err_train, err_test)#Обучение модели и выявление процента ошибки

0.014950436103846772 0.02488026271288144


In [6]:
Bank = rf.predict(df)
Bank = pd.DataFrame(data = Bank, columns=['BANKR'])
writer = pd.ExcelWriter('f.xlsx')
Bank.to_excel(writer,'AR')
writer.save() #Прогнозирование данных и сохранение результатов в лист Excel

NameError: name 'rf' is not defined

In [7]:
Bank1 = gbt.predict(df)
Bank1 = pd.DataFrame(data = Bank1,columns=['BANKR'])
writer = pd.ExcelWriter('GBT.xlsx')
Bank1.to_excel(writer,'AR')
writer.save()  #Прогнозирование данных и сохранение результатов в лист Excel

NameError: name 'gbt' is not defined

In [52]:
Les = pd.read_excel("f.xlsx", decimal = '.')
Bust = pd.read_excel("GBT.xlsx", decimal = '.')
LesPred = Les['BANKR']
BustPred = Bust['BANKR'] #Загрузка результатов

In [53]:
print(classification_report(test,LesPred))  #Метрики оценки для случайного леса

              precision    recall  f1-score   support

           0       1.00      0.86      0.92    730141
           1       0.02      0.80      0.05      3130

    accuracy                           0.86    733271
   macro avg       0.51      0.83      0.48    733271
weighted avg       0.99      0.86      0.92    733271



In [54]:
print(classification_report(test,BustPred)) #Метрики оценки для градиентного бустинга

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    730141
           1       0.17      0.77      0.28      3130

    accuracy                           0.98    733271
   macro avg       0.59      0.88      0.64    733271
weighted avg       1.00      0.98      0.99    733271



In [55]:
cfm = confusion_matrix(test,BustPred) #Вывод матрицы ошибок для градиентного бустинга (TP   FP)
                                                                                     #(FN   TN)
print(cfm)


[[718488  11653]
 [   713   2417]]


In [56]:
cfm1 = confusion_matrix(test,LesPred) #Вывод матрицы ошибок для случайного леса  (TP    FP)
                                                                                #(FN   TN)
print(cfm1)

[[625645 104496]
 [   638   2492]]


In [1]:
from sklearn.metrics import roc_auc_score

In [5]:
print('LDA Auc score = ',roc_auc_score(test,BustPred)) 
print('GNB Auc score = ',roc_auc_score(test,LesPred))

NameError: name 'BustPred' is not defined